In [1]:
import pandas as pd
import numpy as np
import torch

In [4]:
cites = pd.read_csv("cora/cora.cites", sep="\t", names=["node1", "node2"])
cites.head(3)

,node1,node2
0,35,1033
1,35,103482
2,35,103515


In [ ]:
content = pd.read_csv(
    "cora/cora.content",
    sep="\t",
    names=["node"] + [f"feat{i}" for i in range(1433)] + ["label"],
)
content.head(3)

,node,feat0,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,...,feat1424,feat1425,feat1426,feat1427,feat1428,feat1429,feat1430,feat1431,feat1432,label
0,31336,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,Neural_Networks
1,1061127,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,Rule_Learning
2,1106406,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Reinforcement_Learning


In [52]:
content = content.sort_values("node").reset_index(drop=True)
content.head(3)

,node,feat0,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,...,feat1424,feat1425,feat1426,feat1427,feat1428,feat1429,feat1430,feat1431,feat1432,label
0,35,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Genetic_Algorithms
1,40,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Genetic_Algorithms
2,114,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Reinforcement_Learning


In [34]:
# 获取节点和特征
nodes = content["node"].unique()
node2id = {node: i for i, node in enumerate(nodes)}
features = content.iloc[:, 1:-1].values
labels = content["label"].astype("category").cat.codes.values

In [53]:
content["node"].map(node2id).head(3)

0    0
1    1
2    2
Name: node, dtype: int64

In [38]:
# 构建邻接矩阵
edges = cites.copy()
edges["node1"] = cites["node1"].map(node2id)
edges["node2"] = cites["node2"].map(node2id)
adj_matrix = np.zeros((len(nodes), len(nodes)))
adj_matrix[edges["node1"], edges["node2"]] = 1

In [44]:
adj_matrix = torch.zeros((len(nodes), len(nodes)))
adj_matrix[edges["node1"], edges["node2"]] = 1

In [45]:
# 按标签分组并采样
sampled_content = (
    content.groupby("label")
    .apply(lambda x: x.sample(n=20, random_state=42))
    .reset_index(drop=True)
)
sampled_content.head()

/var/folders/_m/z2k5mp3x6l72p84ycrp1kjpc0000gn/T/ipykernel_28488/3561962938.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_content = content.groupby('label').apply(lambda x: x.sample(n=20, random_state=42)).reset_index(drop=True)


,node,feat0,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,...,feat1424,feat1425,feat1426,feat1427,feat1428,feat1429,feat1430,feat1431,feat1432,label
0,170798,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Case_Based
1,1125992,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Case_Based
2,1119987,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Case_Based
3,1953,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Case_Based
4,37483,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,Case_Based


In [54]:
np.random.sample()

array([[[[[9.15050893e-01, 8.71804341e-01, 5.19315019e-02,
           7.34039658e-01, 7.89693299e-01],
          [2.14885279e-01, 2.84023985e-02, 8.39089942e-02,
           2.69684957e-01, 5.88670699e-01],
          [8.82191230e-02, 3.49913959e-01, 1.80857157e-01,
           5.81336001e-01, 9.81852927e-01],
          [3.29010403e-01, 5.99528596e-01, 9.89697936e-01,
           5.62698934e-01, 6.61586113e-01]],

         [[8.59839082e-01, 7.05717692e-01, 5.56025093e-01,
           1.33172458e-01, 3.03349750e-01],
          [3.18623083e-01, 2.91583015e-01, 8.22220404e-01,
           9.38315490e-01, 2.29547510e-01],
          [6.21268197e-01, 3.52308345e-02, 3.48237334e-01,
           7.01987020e-02, 4.56824457e-01],
          [2.05878774e-01, 6.01790569e-01, 2.13857573e-01,
           1.50591203e-01, 6.57759998e-01]],

         [[6.35829915e-01, 2.63548655e-01, 1.19667089e-01,
           4.11876379e-01, 7.83523928e-01],
          [7.96543295e-01, 4.89984932e-01, 6.86784267e-01,
          

In [64]:
a = torch.tensor([0, 0, 1, 0, 1]).float()
b = torch.tensor([0, 0, 1, 1, 2]).float()
c = a.to_sparse() + b.to_sparse()
d = c / torch.norm(c,p=2,dim=-1)
d

tensor(indices=tensor([[2, 3, 4]]),
       values=tensor([0.5345, 0.2673, 0.8018]),
       size=(5,), nnz=3, layout=torch.sparse_coo)

In [66]:
a = (0, )

ValueError: not enough values to unpack (expected 2, got 1)